### jupyter notebook to convert text to ids

author: Jan Jörg
date: 14.03.2024

In [53]:
# imports
import pandas as pd
from db import get_database

In [54]:
dbname = get_database()
collection = dbname["joined"]

documents = collection.find()
df = pd.DataFrame(list(documents))

df.head(2)

,_id,onetsoc19,display,occupation,tasks,technology_skills,tools_used,tools_technology,knowledge,skills,...,job_zone,interests,work_styles,work_values,related_occupations,additional_information,education,2010soc,isco08,Name_de
0,6620dfe3e366d70e7b9073fc,27-2011.00,long,"{'code': '27-2011.00', 'title': 'Actors', 'tag...","{'task': [{'id': 7646, 'green': False, 'relate...",{'category': [{'related': 'https://services.on...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.7.c', 'related': 'htt...","{'element': [{'id': '2.A.1.a', 'related': 'htt...",...,"{'value': 2, 'title': 'Job Zone Two: Some Prep...","{'element': [{'id': '1.B.1.c', 'related': 'htt...","{'element': [{'id': '1.C.3.a', 'related': 'htt...","{'element': [{'id': '1.B.2.d', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actorsequity....,{'level_required': {'category': [{'name': 'Les...,27-2011,2655,Schauspieler
1,6620dfe3e366d70e7b9073fd,15-2011.00,long,"{'code': '15-2011.00', 'title': 'Actuaries', '...","{'task': [{'id': 3500, 'green': False, 'relate...",{'see_all': {'href': 'https://services.onetcen...,{'category': [{'related': 'https://services.on...,{'tools': {'category': [{'related': 'https://s...,"{'element': [{'id': '2.C.4.a', 'related': 'htt...","{'element': [{'id': '2.A.2.a', 'related': 'htt...",...,"{'value': 4, 'title': 'Job Zone Four: Consider...","{'element': [{'id': '1.B.1.f', 'related': 'htt...","{'element': [{'id': '1.C.7.b', 'related': 'htt...","{'element': [{'id': '1.B.2.b', 'related': 'htt...",{'occupation': [{'href': 'https://services.one...,{'source': [{'url': 'https://www.actuary.org/'...,"{'level_required': {'category': [{'name': ""Bac...",15-2011,2120,"Mathematiker, Aktuare und Statistiker"


In [55]:
skills = pd.DataFrame()
abilities = pd.DataFrame()
job_zone = pd.DataFrame()
education = pd.DataFrame()

In [56]:
# Assuming df["skills"] is a column of lists of dictionaries
skills = df['skills'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
skills_df = pd.json_normalize(skills)

# Drop duplicates based on 'id'
skills_df = skills_df.drop_duplicates(subset='id')

# Reset the index
skills_df = skills_df.reset_index(drop=True)

skills_df = skills_df.drop(columns=["related", "score.scale", "score.important", "score.value"])

skills_df['skill_id'] = range(1, len(skills_df) + 1)

skills_df.head(3)

,id,name,description,skill_id
0,2.A.1.a,Reading Comprehension,Understanding written sentences and paragraphs...,1
1,2.A.1.d,Speaking,Talking to others to convey information effect...,2
2,2.A.1.b,Active Listening,Giving full attention to what other people are...,3


In [57]:
dbname = get_database()

collection_name = dbname["skills"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(skills_df.to_dict("records"))

InsertManyResult([ObjectId('662553db621afd872127145b'), ObjectId('662553db621afd872127145c'), ObjectId('662553db621afd872127145d'), ObjectId('662553db621afd872127145e'), ObjectId('662553db621afd872127145f'), ObjectId('662553db621afd8721271460'), ObjectId('662553db621afd8721271461'), ObjectId('662553db621afd8721271462'), ObjectId('662553db621afd8721271463'), ObjectId('662553db621afd8721271464'), ObjectId('662553db621afd8721271465'), ObjectId('662553db621afd8721271466'), ObjectId('662553db621afd8721271467'), ObjectId('662553db621afd8721271468'), ObjectId('662553db621afd8721271469'), ObjectId('662553db621afd872127146a'), ObjectId('662553db621afd872127146b'), ObjectId('662553db621afd872127146c'), ObjectId('662553db621afd872127146d'), ObjectId('662553db621afd872127146e'), ObjectId('662553db621afd872127146f'), ObjectId('662553db621afd8721271470'), ObjectId('662553db621afd8721271471'), ObjectId('662553db621afd8721271472'), ObjectId('662553db621afd8721271473'), ObjectId('662553db621afd87212714

In [58]:
# Assuming df["abilities"] is a column of lists of dictionaries
abilities = df['abilities'].apply(lambda x: x['element']).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
abilities_df = pd.json_normalize(abilities)

# Drop duplicates based on 'id'
abilities_df = abilities_df.drop_duplicates(subset='id')

# Reset the index
abilities_df = abilities_df.reset_index(drop=True)

abilities_df = abilities_df.drop(columns=["related", "score.scale", "score.important" ,"score.value"])

abilities_df['ability_id'] = range(1, len(abilities_df) + 1)

abilities_df.head(3)

,id,name,description,ability_id
0,1.A.1.a.3,Oral Expression,The ability to communicate information and ide...,1
1,1.A.1.a.1,Oral Comprehension,The ability to listen to and understand inform...,2
2,1.A.1.d.1,Memorization,The ability to remember information such as wo...,3


In [59]:
dbname = get_database()

collection_name = dbname["abilities"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(abilities_df.to_dict("records"))

InsertManyResult([ObjectId('662553db621afd872127147f'), ObjectId('662553db621afd8721271480'), ObjectId('662553db621afd8721271481'), ObjectId('662553db621afd8721271482'), ObjectId('662553db621afd8721271483'), ObjectId('662553db621afd8721271484'), ObjectId('662553db621afd8721271485'), ObjectId('662553db621afd8721271486'), ObjectId('662553db621afd8721271487'), ObjectId('662553db621afd8721271488'), ObjectId('662553db621afd8721271489'), ObjectId('662553db621afd872127148a'), ObjectId('662553db621afd872127148b'), ObjectId('662553db621afd872127148c'), ObjectId('662553db621afd872127148d'), ObjectId('662553db621afd872127148e'), ObjectId('662553db621afd872127148f'), ObjectId('662553db621afd8721271490'), ObjectId('662553db621afd8721271491'), ObjectId('662553db621afd8721271492'), ObjectId('662553db621afd8721271493'), ObjectId('662553db621afd8721271494'), ObjectId('662553db621afd8721271495'), ObjectId('662553db621afd8721271496'), ObjectId('662553db621afd8721271497'), ObjectId('662553db621afd87212714

In [74]:
job_zone = pd.DataFrame(list(df["job_zone"]))

# Drop duplicates based on 'value'
job_zone = job_zone.drop_duplicates(subset='value')
job_zone = job_zone.drop(columns=["svp_range", "related_experience"])

job_zone['job_zone_id'] = range(1, len(job_zone) + 1)

job_zone.head(2)

,value,title,education,job_training,job_zone_examples,job_zone_id
0,2,Job Zone Two: Some Preparation Needed,These occupations usually require a high schoo...,Employees in these occupations need anywhere f...,These occupations often involve using your kno...,1
1,4,Job Zone Four: Considerable Preparation Needed,Most of these occupations require a four-year ...,Employees in these occupations usually need se...,Many of these occupations involve coordinating...,2


In [75]:
dbname = get_database()

collection_name = dbname["job_zone"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(job_zone.to_dict("records"))

InsertManyResult([ObjectId('662553e9621afd8721275024'), ObjectId('662553e9621afd8721275025'), ObjectId('662553e9621afd8721275026'), ObjectId('662553e9621afd8721275027'), ObjectId('662553e9621afd8721275028')], acknowledged=True)

In [76]:
# Assuming df["work_values"] is a column of lists of dictionaries
education = df['education'].apply(lambda x: x['level_required']).apply(lambda x : x["category"]).explode().reset_index(drop=True)

# Convert the list of dictionaries into a DataFrame
education_df = pd.json_normalize(education)

# Drop duplicates based on 'name'
education_df = education_df.drop_duplicates(subset='name')
education_df = education_df.drop(columns=["score.scale", "score.value", "description"])

education_df['education_id'] = range(1, len(education_df) + 1)

education_df.head(5)

,name,education_id
0,Less than high school diploma,1
1,Bachelor's degree,2
2,High school diploma or equivalent,3
4,Post-baccalaureate certificate,4
5,Professional degree,5


In [77]:
dbname = get_database()

collection_name = dbname["education"]

# Insert the dictionaries into the MongoDB collection
collection_name.insert_many(education_df.to_dict("records"))

InsertManyResult([ObjectId('662553e9621afd872127502a'), ObjectId('662553e9621afd872127502b'), ObjectId('662553e9621afd872127502c'), ObjectId('662553e9621afd872127502d'), ObjectId('662553e9621afd872127502e'), ObjectId('662553e9621afd872127502f'), ObjectId('662553e9621afd8721275030'), ObjectId('662553e9621afd8721275031'), ObjectId('662553e9621afd8721275032'), ObjectId('662553e9621afd8721275033'), ObjectId('662553e9621afd8721275034'), ObjectId('662553e9621afd8721275035')], acknowledged=True)